# ACC calculation - 6 hourly IFS

In [1]:
import os
import sys
import yaml
import argparse
from glob import glob
from datetime import datetime, timedelta

import numpy as np
import xarray as xr

sys.path.insert(0, os.path.realpath('../libs/'))
import verif_utils as vu

### Load config

In [2]:
config_name = os.path.realpath('verif_config.yml')

with open(config_name, 'r') as stream:
    conf = yaml.safe_load(stream)

### Model and lead time to verify

In [3]:
model_name = 'IFS'
lead_range = conf[model_name]['lead_range']
verif_lead_range = conf[model_name]['verif_lead_range']

leads_exist = list(np.arange(lead_range[0], lead_range[-1]+lead_range[0], lead_range[0]))
leads_verif = list(np.arange(verif_lead_range[0], verif_lead_range[-1]+verif_lead_range[0], verif_lead_range[0]))
ind_lead = vu.lead_to_index(leads_exist, leads_verif)

print('Verifying lead times: {}'.format(leads_verif))
print('Verifying lead indices: {}'.format(ind_lead))

Verifying lead times: [6, 12, 18, 24, 30, 36, 42, 48, 54, 60, 66, 72, 78, 84, 90, 96, 102, 108, 114, 120, 126, 132, 138, 144, 150, 156, 162, 168, 174, 180, 186, 192, 198, 204, 210, 216, 222, 228, 234, 240]
Verifying lead indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [4]:
verif_ind_start = 0; verif_ind_end = 2*(365+366+365)

path_verif = conf[model_name]['save_loc_verif']+'combined_acc_{}_{}_{}h_{}h_{}.nc'.format(
    verif_ind_start, verif_ind_end, verif_lead_range[0], verif_lead_range[-1], model_name)

# ---------------------------------------------------------------------------------------- #
# temp function for lat / lon average
def sp_avg(DS, wlat):
    return DS.weighted(wlat).mean(['latitude', 'longitude'], skipna=False)

### ACC compute

In [5]:
# ---------------------------------------------------------------------------------------- #
# ERA5 climatology info
ERA5_path_string = conf['ERA5_weatherbench']['save_loc_clim'] + 'ERA5_clim_1990_2019_6h_interp.nc'
ds_ERA5_clim = xr.open_dataset(ERA5_path_string)

# rename coordinate name to latitude, longitude
if 'lat' in ds_ERA5_clim.coords:
    ds_ERA5_clim = ds_ERA5_clim.rename({'lat': 'latitude'})
if 'lon' in ds_ERA5_clim.coords:
    ds_ERA5_clim = ds_ERA5_clim.rename({'lon': 'longitude'})

# ---------------------------------------------------------------------------------------- #
# ERA5 verif target
filename_ERA5 = sorted(glob(conf['ERA5_ours']['save_loc']))

# pick years
year_range = conf['ERA5_ours']['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_ERA5 = [fn for fn in filename_ERA5 if any(year in fn for year in years_pick)]

# merge yearly ERA5 as one
ds_ERA5 = [vu.get_forward_data(fn) for fn in filename_ERA5]
ds_ERA5_merge = xr.concat(ds_ERA5, dim='time')
    
# Select the specified variables and their levels
variables_levels = conf['ERA5_ours']['verif_variables']

# subset merged ERA5
ds_ERA5_merge = vu.ds_subset_everything(ds_ERA5_merge, variables_levels)

# ---------------------------------------------------------------------------------------- #
# forecast
filename_OURS = sorted(glob(conf[model_name]['save_loc_gather']+'*.nc'))

# pick years
year_range = conf[model_name]['year_range']
years_pick = np.arange(year_range[0], year_range[1]+1, 1).astype(str)
filename_OURS = [fn for fn in filename_OURS if any(year in fn for year in years_pick)]

L_max = len(filename_OURS)
print('Total number of initialization times: {}'.format(L_max))
assert verif_ind_end <= L_max, 'verified indices (days) exceeds the max index available'

filename_OURS = filename_OURS[verif_ind_start:verif_ind_end]

# ---------------------------------------------------------------------------------------- #
# latitude weighting
lat = xr.open_dataset(filename_OURS[0])["lat"]
w_lat = np.cos(np.deg2rad(lat))
w_lat = w_lat / w_lat.mean()
w_lat = w_lat.rename({'lat': 'latitude'})

# ---------------------------------------------------------------------------------------- #
# ACC compute

acc_results = []

for fn_ours in filename_OURS:
    
    # --------------------------------------------------------------- #
    # import and subset forecast
    ds_ours = xr.open_dataset(fn_ours)
    ds_ours = vu.ds_subset_everything(ds_ours, variables_levels)
    ds_ours = ds_ours.isel(time=ind_lead)
    dayofyear = ds_ours['time.dayofyear']
    hourofday = ds_ours['time'].dt.hour
    ds_ours = ds_ours.load()

    # rename coordinate name to latitude, longitude
    if 'lat' in ds_ours.coords:
        ds_ours = ds_ours.rename({'lat': 'latitude'})
    if 'lon' in ds_ours.coords:
        ds_ours = ds_ours.rename({'lon': 'longitude'})
    
    # --------------------------------------------------------------- #
    # get ERA5 verification target
    ds_target = ds_ERA5_merge.sel(time=ds_ours['time']).load()
    
    # --------------------------------------------------------------- #
    # get ERA5 climatology
    ds_clim_target = ds_ERA5_clim.sel(dayofyear=dayofyear, hour=hourofday).load()
    
    # ========================================== #
    # ERA5 anomaly
    ds_anomaly_ERA5 = ds_target - ds_clim_target

    # fcst anomaly
    ds_anomaly_OURS = ds_ours - ds_clim_target
    
    # ========================================== #
    # ACC with latitude-based cosine weights (check sp_avg and w_lat)
    top = sp_avg(ds_anomaly_OURS*ds_anomaly_ERA5, w_lat)
    
    bottom = np.sqrt(
        sp_avg(ds_anomaly_OURS**2, w_lat) * sp_avg(ds_anomaly_ERA5**2, w_lat))
                
    acc_results.append((top/bottom).drop_vars('time'))
    
# Combine ACC results
ds_acc = xr.concat(acc_results, dim='days')

# # Save
# print('Save to {}'.format(path_verif))
# ds_acc.to_netcdf(path_verif)

Total number of initialization times: 2922


In [6]:
ds_acc['Z500']

<xarray.DataArray 'Z500' (days: 3, time: 40)>
array([[0.99928339, 0.99879959, 0.99880545, 0.99791549, 0.99755634,
        0.99626518, 0.9955086 , 0.9933177 , 0.99159153, 0.98919866,
        0.98717483, 0.98260051, 0.97726155, 0.97032383, 0.96347952,
        0.95517298, 0.94748284, 0.93701936, 0.9246091 , 0.90787636,
        0.89299339, 0.87787492, 0.86594826, 0.85379158, 0.84106712,
        0.82316611, 0.80209989, 0.7794527 , 0.75525564, 0.73159718,
        0.70551804, 0.67899853, 0.64436039, 0.6028117 , 0.55178011,
        0.4977795 , 0.44413158, 0.39198841, 0.34999104, 0.3136591 ],
       [0.99942125, 0.99888708, 0.99872834, 0.9976853 , 0.99725249,
        0.99559654, 0.99474289, 0.99291474, 0.99126906, 0.98823157,
        0.98588547, 0.98200388, 0.97851837, 0.97270086, 0.96675849,
        0.9581011 , 0.94797429, 0.93221189, 0.91464609, 0.89516758,
        0.87930469, 0.86414365, 0.84944468, 0.83706138, 0.82423312,
        0.80734074, 0.78546872, 0.75817008, 0.72671569, 0.69064019,
        0.6566435 , 0.61566408, 0.56821037, 0.51792841, 0.47640055,
        0.4468976 , 0.42609424, 0.40269212, 0.37726088, 0.35137104],
       [0.99946118, 0.99889623, 0.99886454, 0.99784584, 0.99756458,
        0.99633618, 0.99559402, 0.99369355, 0.99231282, 0.98970692,
        0.98785986, 0.98339996, 0.97963257, 0.97286492, 0.96682561,
        0.95650024, 0.94334508, 0.92560761, 0.90985402, 0.89250287,
        0.87409078, 0.85454556, 0.82993247, 0.79962637, 0.77227238,
        0.74486953, 0.71824921, 0.69481135, 0.67411483, 0.6463618 ,
        0.61154849, 0.57109675, 0.5356968 , 0.49724635, 0.46040032,
        0.42212198, 0.38056653, 0.33984628, 0.30584586, 0.28336637]])
Coordinates:
    dayofyear  (days, time) int64 1 1 1 2 2 2 2 3 3 ... 10 10 10 11 11 11 11 12
    hour       (days, time) int64 6 12 18 0 6 12 18 0 6 ... 6 12 18 0 6 12 18 0
Dimensions without coordinates: days, time